In [ ]:
from IPython.display import display, HTML

In [ ]:
html_code = """
<canvas id="pendulumCanvas" width="400" height="300"></canvas>
<button onclick="startAnimation()">Start</button>
<button onclick="pauseAnimation()">Pause</button>

<script>
let canvas = document.getElementById("pendulumCanvas");
let ctx = canvas.getContext("2d");
let playing = false;
let time = 0;

function drawPendulum(x, y) {
    ctx.clearRect(0, 0, canvas.width, canvas.height);
    let originX = canvas.width / 2, originY = 50;
    ctx.beginPath();
    ctx.moveTo(originX, originY);
    ctx.lineTo(originX + x * 100, originY + y * 100);
    ctx.stroke();
    ctx.beginPath();
    ctx.arc(originX + x * 100, originY + y * 100, 10, 0, Math.PI * 2);
    ctx.fill();
}

function update() {
    if (!playing) return;
    fetch('http://127.0.0.1:5000/calculate', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ time: time })
    })
    .then(response => response.json())
    .then(data => {
        console.log(data);
        drawPendulum(data.x, data.y);
        time += 0.05;
        requestAnimationFrame(update);
    });
}

function startAnimation() { playing = true; update(); }
function pauseAnimation() { playing = false; }
</script>

<script>
console.log("JavaScript Loaded!");

</script>
"""

display(HTML(html_code))

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Animated GDP and population over decades'),
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=["GDP - Scatter", "Population - Bar"],
        value='GDP - Scatter',
    ),
    dcc.Loading(dcc.Graph(id="graph"), type="cube")
])


@app.callback(
    Output("graph", "figure"),
    Input("selection", "value"))
def display_animated_graph(selection):
    df = px.data.gapminder() # replace with your own data source
    animations = {
        'GDP - Scatter': px.scatter(
            df, x="gdpPercap", y="lifeExp", animation_frame="year",
            animation_group="country", size="pop", color="continent",
            hover_name="country", log_x=True, size_max=55,
            range_x=[100,100000], range_y=[25,90]),
        'Population - Bar': px.bar(
            df, x="continent", y="pop", color="continent",
            animation_frame="year", animation_group="country",
            range_y=[0,4000000000]),
    }
    return animations[selection]


app.run(debug=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0.01, 10, 1000)
plt.plot(x, np.log(x))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0.2, 6000, 1000)
plt.plot(x, np.exp(1/x))
print (np.exp(1/x[-1]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0.2, 20, 1000)
plt.plot(x, np.log(x) + np.exp(1/x))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0.2, 20, 1000)
plt.plot(x, -(-np.exp(1/x)/x**2) + 1/x)

In [ ]:
from sympy import *

In [ ]:
x = symbols('x')
u = exp(1/x) + log(x)
u

In [ ]:
f = -u.diff(x)
f

In [ ]:
nsolve(f, x, 1)

In [ ]:
import numpy as np

# Define the radial force function
def f(r):
    # Example: Inverse-square law force
    return -1/r + np.exp(1/r)/r**2 if r != 0 else 0

# Define the system of ODEs for two particles
def system(t, y):
    # y = [x1, y1, vx1, vy1, x2, y2, vx2, vy2]
    x1, y1, vx1, vy1, x2, y2, vx2, vy2 = y
    
    # Calculate distances
    dx = x2 - x1
    dy = y2 - y1
    r = np.sqrt(dx**2 + dy**2)
    
    # Forces
    force = f(r)
    fx = force * dx / r
    fy = force * dy / r
    
    # Derivatives
    dx1dt = vx1
    dy1dt = vy1
    dvx1dt = fx
    dvy1dt = fy
    
    dx2dt = vx2
    dy2dt = vy2
    dvx2dt = -fx
    dvy2dt = -fy
    
    return np.array([dx1dt, dy1dt, dvx1dt, dvy1dt, dx2dt, dy2dt, dvx2dt, dvy2dt])

# Runge-Kutta 4th order method
def runge_kutta_4(func, t0, y0, t_end, dt):
    t = t0
    y = y0
    trajectory = [y0]
    times = [t0]
    
    while t < t_end:
        k1 = func(t, y)
        k2 = func(t + dt / 2, y + dt / 2 * k1)
        k3 = func(t + dt / 2, y + dt / 2 * k2)
        k4 = func(t + dt, y + dt * k3)
        y = y + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        t += dt
        trajectory.append(y)
        times.append(t)
    
    return np.array(times), np.array(trajectory)

# Initial conditions
t0 = 0
t_end = 60
dt = 0.01
y0 = np.array([1, 0, 0, 0, -1, 0, 0, 0])  # [x1, y1, vx1, vy1, x2, y2, vx2, vy2]

# Simulate
times, trajectory = runge_kutta_4(system, t0, y0, t_end, dt)

# Extract positions
x1, y1 = trajectory[:, 0], trajectory[:, 1]
x2, y2 = trajectory[:, 4], trajectory[:, 5]

# Plot the trajectories
import matplotlib.pyplot as plt

plt.plot(times, x1, label="Particle 1")
plt.plot(times, x2, label="Particle 2")
plt.plot(times, y1, label="Particle 1")
plt.plot(times, y2, label="Particle 2")
plt.xlabel("t")
plt.ylabel("x")
plt.legend()
plt.title("Trajectories of Two Particles")
plt.axis("equal")
plt.show()

In [ ]:
x1.mean() - x2.mean()